# Importing python packages

In [ ]:
import glob
from multiprocessing import Pool
import time
from tqdm import tqdm

# Importing SSPARQ setup

In [ ]:
from parameters_py.config import (
					WAVEFORM_DIR,CATALOG_FILE,XML_DIR,SSPARQ_OUTPUT,num_processes
				   )

# Importing SSPARQ functions

In [ ]:
from src.analysis import (
					aic_simple,find_orientation,Braunmiller_Pornsopin_algorithm,calculate_metrics
				   )

from src.utils import (
					quakeml_to_dataframe,moment_tensor_to_nodal_planes,calculate_plunge,mecclass,adjust_baz_for_ZEN,rms,energy
				   )

# Main program

## Example I: One station

### Reading catalog

In [ ]:
cat = quakeml_to_dataframe(CATALOG_FILE)
cat.tail(2)

### Reading station

In [ ]:
STATIONS_xml = sorted(glob.glob(XML_DIR+'*'))[0]

In [ ]:
STATIONS_xml

### Creating input list for one station

In [ ]:
input_list = [STATIONS_xml,WAVEFORM_DIR,cat,SSPARQ_OUTPUT]

### Running multiprocessing function given a input list

In [ ]:
start_time = time.time()

calculate_metrics(input_list)

print('\n')
print("--- %.2f execution time (min) ---" % ((time.time() - start_time)/60))
print('\n')

## Example II: Multiple stations (determined by the number of files in the XML_DIR)

### Reading catalog

In [ ]:
cat = quakeml_to_dataframe(CATALOG_FILE)
cat.tail(2)

### Reading station

In [ ]:
STATIONS_xml = sorted(glob.glob(XML_DIR+'*'))

In [ ]:
STATIONS_xml

### Creating input list for one station

In [ ]:
input_list = []
for xml_file in STATIONS_xml:
    input_list.append([xml_file,WAVEFORM_DIR,cat,SSPARQ_OUTPUT])

In [ ]:
print("Number of stations:",len(input_list))

### Running multiprocessing function given a input list

In [ ]:
start_time = time.time()

for sta_input in input_list:
    calculate_metrics(sta_input)

print('\n')
print("--- %.2f execution time (min) ---" % ((time.time() - start_time)/60))
print('\n')